In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from pandas.plotting import scatter_matrix
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [5]:
train = pd.read_csv('data/Train_O4UPEyW.csv')

In [6]:
train['ZONEID'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [7]:
#convert timestamp to datetime
train['TIMESTAMP'] = pd.to_datetime(train['TIMESTAMP'])

In [8]:
deg_per_rad = 180/math.pi

In [9]:
#calc wind direction --> atan2(-Ugeo,-Vgeo) * DperR
#calculate wind direction and speed

In [10]:
train['wind_dir_10'] = 270 - (np.arctan2(-train['U10'], -train['V10']) * deg_per_rad)
train['wind_dir_100'] = 270 - (np.arctan2(-train['U100'], -train['V100']) * deg_per_rad)
train['wind_dir_10'] = [dir - 360 if dir > 180 else dir for dir in train['wind_dir_10']]
train['wind_dir_100'] = [dir - 360 if dir > 180 else dir for dir in train['wind_dir_100']]

In [11]:
train['wind_spd_10'] = np.sqrt(train['U10']**2 + train['V10']**2)
train['wind_spd_100'] = np.sqrt(train['U100']**2 + train['V100']**2)

In [12]:
train['U10_sqr'] = train['U10'] **2
train['V10_sqr'] = train['V10'] **2
train['U100_sqr'] = train['U100'] **2
train['V100_sqr'] = train['V100'] **2

In [13]:
#create month column to user for getting q1-4 groupings
train['month'] = train['TIMESTAMP'].apply(lambda x: x.month)

In [14]:
train['q1'] = [1 if month < 4 else 0 for month in train['month']]
train['q2'] = [1 if 3 < month < 7 else 0 for month in train['month']]
train['q3'] = [1 if 6 < month < 10 else 0 for month in train['month']]
train['q4'] = [1 if 9 < month else 0 for month in train['month']]

In [15]:
#creat dummy columns for grouped time of day
train['day_q1'] = [1 if time < 6 else 0 for time in train['TIMESTAMP'].apply(lambda x: x.hour)]
train['day_q2'] = [1 if 5 < time < 12 else 0 for time in train['TIMESTAMP'].apply(lambda x: x.hour)]
train['day_q3'] = [1 if 11 < time < 18 else 0 for time in train['TIMESTAMP'].apply(lambda x: x.hour)]
train['day_q4'] = [1 if time > 17 else 0 for time in train['TIMESTAMP'].apply(lambda x: x.hour)]

In [16]:
train = train.drop('month', axis = 1)

In [17]:
def feature_eng(df):
    deg_per_rad = 180/math.pi
    #convert timestamp to datetime
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    
    #get geographic wind direction
    df['wind_dir_10'] = 270 - (np.arctan2(-df['U10'], -df['V10']) * deg_per_rad)
    df['wind_dir_100'] = 270 - (np.arctan2(-df['U100'], -df['V100']) * deg_per_rad)
    df['wind_dir_10'] = [dir - 360 if dir > 180 else dir for dir in df['wind_dir_10']]
    df['wind_dir_100'] = [dir - 360 if dir > 180 else dir for dir in df['wind_dir_100']]
    
    #get wind speeds
    df['wind_spd_10'] = np.sqrt(df['U10']**2 + df['V10']**2)
    df['wind_spd_100'] = np.sqrt(df['U100']**2 + df['V100']**2)
    
    #create ^2 values
    df['U10_sqr'] = df['U10'] **2
    df['V10_sqr'] = df['V10'] **2
    df['U100_sqr'] = df['U100'] **2
    df['V100_sqr'] = df['V100'] **2
    
    #create month column to user for getting q1-4 groupings
    df['month'] = df['TIMESTAMP'].apply(lambda x: x.month)
    
    df['q1'] = [1 if month < 4 else 0 for month in df['month']]
    df['q2'] = [1 if 3 < month < 7 else 0 for month in df['month']]
    df['q3'] = [1 if 6 < month < 10 else 0 for month in df['month']]
    df['q4'] = [1 if 9 < month else 0 for month in df['month']]
    
    #creat dummy columns for grouped time of day
    df['day_q1'] = [1 if time < 6 else 0 for time in df['TIMESTAMP'].apply(lambda x: x.hour)]
    df['day_q2'] = [1 if 5 < time < 12 else 0 for time in df['TIMESTAMP'].apply(lambda x: x.hour)]
    df['day_q3'] = [1 if 11 < time < 18 else 0 for time in df['TIMESTAMP'].apply(lambda x: x.hour)]
    df['day_q4'] = [1 if time > 17 else 0 for time in df['TIMESTAMP'].apply(lambda x: x.hour)]
    
    df = df.drop('month', axis = 1)
    
    return df

In [18]:
df = feature_eng(train)

In [19]:
def divide_zones(df):
    all_zones = []
    for zone in df['ZONEID'].unique():
        different_zone = df[df['ZONEID'] == zone]
        all_zones.append(different_zone)
    return all_zones

In [20]:
my_zones = divide_zones(train)

In [21]:
zone_1_targ = my_zones[8]['TARGETVAR']
zone_1_X = my_zones[8].drop(['TARGETVAR', 'TIMESTAMP', 'ZONEID', 'ID'], axis = 1)

In [22]:
rf = RandomForestRegressor()
gb = GradientBoostingRegressor()
lin = Lasso()

rf_params = {'n_estimators': [10, 100, 500, 1000], 'n_jobs': [-1], 'max_features':[0.5, 0.6, 0.7, 0.8, 0.9]}
gb_params = {'learning_rate': [0.01, 0.1, 0.3, 0.5], 'n_estimators': [50, 100, 250, 500], 'max_depth': [2,3,5,10]}
lin_params = {'alpha':[0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0]}

regressors = [rf, gb, lin]
reg_params = [rf_params, gb_params, lin_params]

In [23]:
def find_best_regressor(df, regressors, parameters):
    score = 100
    for i in xrange(len(regressors)):
        clf = GridSearchCV(regressors[i], parameters[i], scoring='neg_mean_squared_error', verbose=5, n_jobs=-1)
        X = df.drop(['TARGETVAR', 'TIMESTAMP', 'ZONEID', 'ID'], axis=1)
        y = df['TARGETVAR']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
        clf.fit(X_train, y_train)
        prediction = clf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, prediction))
        print "root mean squared error: ", rmse
        if rmse < score:
            score = rmse
            estimator = clf.best_estimator_
            params = clf.best_params_
    
    return estimator, params
        

In [459]:
estimator, params = find_best_regressor(my_zones[0], regressors, reg_params)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] max_features=0.5, n_estimators=10, n_jobs=-1 ....................
[CV] max_features=0.5, n_estimators=10, n_jobs=-1 ....................
[CV] max_features=0.5, n_estimators=10, n_jobs=-1 ....................
[CV] max_features=0.5, n_estimators=100, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=100, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=100, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=500, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=500, n_jobs=-1 ...................
[CV]  max_features=0.5, n_estimators=10, n_jobs=-1, score=-0.0286245970411, total=   0.6s
[CV] max_features=0.5, n_estimators=500, n_jobs=-1 ...................
[CV]  max_features=0.5, n_estimators=10, n_jobs=-1, score=-0.0287683521851, total=   0.6s
[CV]  max_features=0.5, n_estimators=10, n_jobs=-1, score=-0.0281144851534, total=   0.6s
[CV] max_features=0.5, n_estim

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s


[CV]  max_features=0.5, n_estimators=100, n_jobs=-1, score=-0.0259131045218, total=   3.7s
[CV] max_features=0.5, n_estimators=1000, n_jobs=-1 ..................
[CV]  max_features=0.5, n_estimators=100, n_jobs=-1, score=-0.0263042096476, total=   3.8s
[CV] max_features=0.6, n_estimators=10, n_jobs=-1 ....................
[CV]  max_features=0.5, n_estimators=100, n_jobs=-1, score=-0.0256655099503, total=   3.8s
[CV] max_features=0.6, n_estimators=10, n_jobs=-1 ....................
[CV]  max_features=0.6, n_estimators=10, n_jobs=-1, score=-0.0282042040898, total=   0.6s
[CV]  max_features=0.6, n_estimators=10, n_jobs=-1, score=-0.0290056850247, total=   0.5s
[CV] max_features=0.6, n_estimators=10, n_jobs=-1 ....................
[CV] max_features=0.6, n_estimators=100, n_jobs=-1 ...................
[CV]  max_features=0.6, n_estimators=10, n_jobs=-1, score=-0.0278667124929, total=   0.8s
[CV] max_features=0.6, n_estimators=100, n_jobs=-1 ...................
[CV]  max_features=0.6, n_estim

[CV]  max_features=0.9, n_estimators=500, n_jobs=-1, score=-0.025778925224, total=  59.1s
[CV]  max_features=0.8, n_estimators=1000, n_jobs=-1, score=-0.0256101804977, total= 1.6min
[CV]  max_features=0.9, n_estimators=1000, n_jobs=-1, score=-0.0258685159125, total= 1.5min


[Parallel(n_jobs=-1)]: Done  58 out of  60 | elapsed:  4.9min remaining:   10.0s


...

root mean squared error:  0.158968135818
Fitting 3 folds for each of 7 candidates, totalling 21 fits
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.01 ......................................................
[CV] alpha=0.01 ......................................................
[CV] alpha=0.01 ......................................................
[CV] alpha=0.1 .......................................................
[CV] alpha=0.1 .......................................................
[CV] ................ alpha=0.1, score=-0.0368869619119, total=   0.0s
[CV] ............... alpha=0.01, score=-0.0359626040919, total=   0.1s
[CV] alpha=0.1 .......................................................
[CV] ................ alpha=0.1, score=-0.0342573134627, total=   0.0s
[CV] alpha=0.25 ...............................

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  11 out of  21 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  16 out of  21 | elapsed:    0.4s remaining:    0.1s


[CV] .............. alpha=0.001, score=-0.0339429412264, total=   0.4s
[CV] .............. alpha=0.001, score=-0.0323954729887, total=   0.4s
[CV] .............. alpha=0.001, score=-0.0322657008496, total=   0.4s


[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.5s finished


root mean squared error:  0.179128541005


In [463]:
estimator

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.6, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [464]:
params

{'max_features': 0.6, 'n_estimators': 500, 'n_jobs': -1}

In [24]:
test = pd.read_csv('data/Test_uP7dymh.csv')

In [25]:
def regressor_per_zone(all_zones):
    rf = RandomForestRegressor()
    gb = GradientBoostingRegressor()
    lin = Lasso()

    rf_params = {'n_estimators': [10, 100, 500, 1000], 'n_jobs': [-1], 'max_features':[0.5, 0.6, 0.7, 0.8, 0.9]}
    gb_params = {'learning_rate': [0.01, 0.1, 0.3, 0.5], 'n_estimators': [50, 100, 250, 500], 'max_depth': [2,3,5,10]}
    lin_params = {'alpha':[0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0]}

    regressors = [rf, gb, lin]
    reg_params = [rf_params, gb_params, lin_params]
    
    zone_estimators = []
    zone_params = []
    for i in xrange(len(all_zones)):
        estimator, params = find_best_regressor(all_zones[i], regressors, reg_params)
        zone_estimators.append(estimator)
        zone_params.append(params)
    
    return zone_estimators, zone_params

In [26]:
estimators, params = regressor_per_zone(my_zones)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] max_features=0.5, n_estimators=10, n_jobs=-1 ....................
[CV] max_features=0.5, n_estimators=10, n_jobs=-1 ....................
[CV] max_features=0.5, n_estimators=10, n_jobs=-1 ....................
[CV] max_features=0.5, n_estimators=100, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=100, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=100, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=500, n_jobs=-1 ...................
[CV] max_features=0.5, n_estimators=500, n_jobs=-1 ...................
[CV]  max_features=0.5, n_estimators=10, n_jobs=-1, score=-0.0267776075562, total=   0.5s
[CV] max_features=0.5, n_estimators=500, n_jobs=-1 ...................
[CV]  max_features=0.5, n_estimators=10, n_jobs=-1, score=-0.0280363606483, total=   0.6s
[CV]  max_features=0.5, n_estimators=10, n_jobs=-1, score=-0.0265479671051, total=   0.6s
[CV] max_features=0.5, n_estim

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s


[CV]  max_features=0.5, n_estimators=100, n_jobs=-1, score=-0.0242316916773, total=   3.3s
[CV] max_features=0.5, n_estimators=1000, n_jobs=-1 ..................
[CV]  max_features=0.5, n_estimators=100, n_jobs=-1, score=-0.0246127383246, total=   3.4s
[CV] max_features=0.6, n_estimators=10, n_jobs=-1 ....................
[CV]  max_features=0.5, n_estimators=100, n_jobs=-1, score=-0.0256108604969, total=   3.4s
[CV] max_features=0.6, n_estimators=10, n_jobs=-1 ....................
[CV]  max_features=0.6, n_estimators=10, n_jobs=-1, score=-0.0271076032143, total=   0.5s
[CV] max_features=0.6, n_estimators=10, n_jobs=-1 ....................
[CV]  max_features=0.6, n_estimators=10, n_jobs=-1, score=-0.0278137020459, total=   0.5s
[CV] max_features=0.6, n_estimators=100, n_jobs=-1 ...................
[CV]  max_features=0.6, n_estimators=10, n_jobs=-1, score=-0.0263635217565, total=   0.8s
[CV] max_features=0.6, n_estimators=100, n_jobs=-1 ...................
[CV]  max_features=0.6, n_estim

[CV]  max_features=0.9, n_estimators=500, n_jobs=-1, score=-0.024077420084, total=  59.2s
[CV]  max_features=0.8, n_estimators=1000, n_jobs=-1, score=-0.0239720342883, total= 1.6min
[CV]  max_features=0.9, n_estimators=1000, n_jobs=-1, score=-0.0246215017191, total= 1.5min


[Parallel(n_jobs=-1)]: Done  58 out of  60 | elapsed:  4.8min remaining:    9.9s


...

[CV]  max_features=0.9, n_estimators=500, n_jobs=-1, score=-0.0140541110082, total=  57.3s
[CV]  max_features=0.8, n_estimators=1000, n_jobs=-1, score=-0.0139890208733, total= 1.5min
[CV]  max_features=0.9, n_estimators=1000, n_jobs=-1, score=-0.0154798984298, total= 1.4min


[Parallel(n_jobs=-1)]: Done  58 out of  60 | elapsed:  4.7min remaining:    9.6s


[CV]  max_features=0.9, n_estimators=1000, n_jobs=-1, score=-0.0159336474796, total= 1.3min


Process PoolWorker-32:
Process PoolWorker-31:
Process PoolWorker-29:
Process PoolWorker-30:
Process PoolWorker-27:
Process PoolWorker-26:
Process PoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/npng/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/Users/npng/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/npng/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/npng/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/npng/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/npng/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/npn

KeyboardInterrupt: 